In [85]:
import utils
from sklearn.preprocessing import LabelEncoder

features, player_info = utils.load_player_statistics()

mask = (player_info["Matches Played"] > 8) & (player_info["Playing Time_Min"] > 60)
player_info = player_info[mask]
features = features[mask]

playing_time_cols = ['Playing Time_Minutes', 'Playing Time_Mn/MP','Starts', 'Mn/Start', 'Compl',
                     'Subs', 'unSub', 'PPM','onG', 'onGA','On-Off'
                    ]
col_to_drop = playing_time_cols

X = features.drop(columns = col_to_drop)
y = player_info["Global Pos"]

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model = SVC().fit(X_train, y_train)

In [87]:
model.score(X_test, y_test)

0.7830508474576271